In [2]:
import numpy as np
import pickle
import tqdm
import multiprocess
import json
import glob
import scipy.stats
import sys
sys.path.append('..')
from common import Braid, phi

## MPO Data

In [2]:
orig_braids = pickle.load(open("raw_data/mpo_data_d0.pkl", "rb"))

In [3]:
orig_braids2 = pickle.load(open("raw_data/braids_n10_29_l5_50_c50_d8_1024.pkl", "rb"))

In [4]:
braids = [
    {**a, **b} for a, b in zip(orig_braids, orig_braids2)
]

In [5]:
def calculate_depth(strands, clist):
    depths = [0] * strands
    for idx, _ in clist:
        depths[idx] = depths[idx + 1] = max(depths[idx], depths[idx + 1]) + 1
    return max(depths)

In [6]:
crossings = np.array([len(b['crossing_list']) for b in braids])
qubits = np.array([b['strands'] + 1 for b in braids])

abs_errs = np.array([
    [abs(b['mpo-0']['estimate'] - b[k]) for b in braids] for k in ['mpo-8', 'mpo-16', 'mpo-32', 'mpo-64', 'mpo-128', 'mpo-256', 'mpo-512', 'mpo-1024']
])

rel_errs = np.array([
    [abs(b['mpo-0']['estimate'] - b[k])/abs(b['mpo-0']['estimate']) for b in braids] for k in ['mpo-8', 'mpo-16', 'mpo-32', 'mpo-64', 'mpo-128', 'mpo-256', 'mpo-512', 'mpo-1024']
])

max_chi = np.array([max(max(s) for s in b['mpo-0']['bond_sizes']) for b in braids])

max_intermediate = np.array([
    max(
        4 * (cs[0] + cs[-1] + sum(cs[i] * cs[i + 1] for i in range(len(cs) - 1)))
        for cs in b['mpo-0']['bond_sizes']
    )
    for b in braids
])

depths = np.array([calculate_depth(r['strands'], r['crossing_list']) for r in braids])

In [22]:
normalized_rel_errs = np.concatenate([
    np.stack([8*2**i / max_chi, rel_errs[i, :]], axis=1)
    for i in range(8)
], axis=0)

normalized_rel_errs_capped = normalized_rel_errs[normalized_rel_errs[:, 0] <= 1.0]

In [30]:
def compute_max_intermediate_for_chi(idx):
    b = braids[idx]
    mchi = max_chi[idx]
    return [max(
        4 * (min(cs[0], mchi*f) + min(cs[-1], mchi*f) + sum(min(cs[i], mchi*f) * min(cs[i + 1], mchi*f) for i in range(len(cs) - 1)))
        for cs in b['mpo-0']['bond_sizes']
    ) for f in np.linspace(0.0, 1.0, 20)]

if __name__ == "__main__":
    pool = multiprocess.Pool(64) 
    max_intermediate_chi = np.array(list(tqdm.tqdm(pool.imap(compute_max_intermediate_for_chi, range(len(braids))), total=len(braids))))

100%|██████████| 10000/10000 [00:23<00:00, 420.12it/s]


In [74]:
def compute_max_intermediate_for_chi2(idx):
    b = braids[idx]
    return [max(
        4 * (min(cs[0], f) + min(cs[-1], f) + sum(min(cs[i], f) * min(cs[i + 1], f) for i in range(len(cs) - 1)))
        for cs in b['mpo-0']['bond_sizes']
    ) for f in [8, 16, 32, 64, 128, 256, 512, 1024]]

if __name__ == "__main__":
    pool = multiprocess.Pool(64) 
    max_intermediate_chi2 = np.array(list(tqdm.tqdm(pool.imap(compute_max_intermediate_for_chi2, range(len(braids))), total=len(braids))))

100%|██████████| 10000/10000 [00:05<00:00, 1938.70it/s]


In [77]:
normalized_rel_errsa = np.concatenate([
    np.stack([max_intermediate_chi2[:, i] / max_intermediate, rel_errs[i, :]], axis=1)
    for i in range(8)
], axis=0)

normalized_rel_errsa_capped = normalized_rel_errsa[normalized_rel_errsa[:, 0] <= 1.0]

In [26]:
def canonicalize_cost(dims, idx):
    chi0 = 1 if idx == 0 else dims[idx - 1]
    chi1 = dims[idx]
    chi2 = 1 if idx == len(dims) - 1 else dims[idx + 1]
    return chi1**2 * (8*chi0 + 8*chi2 - 2*chi1 / 3)

def multiply_cost(dims, idx, k=3):
    if k == 3:
        chi0 = 1 if idx == 0 else dims[idx - 1]
        chi1 = dims[idx]
        chi2 = dims[idx + 1]
        chi3 = 1 if idx == len(dims) - 2 else dims[idx + 2]
        return 32 * (chi0*chi1 + 2 * chi1*chi2 + chi2*chi3)
    elif k == 2:
        chi0 = 1 if idx == 0 else dims[idx - 1]
        chi1 = dims[idx]
        chi2 = 1 if idx == len(dims) - 1 else dims[idx + 1]
        return 32 * (chi0*chi1 + chi1*chi2)

def compress_cost(dims, dims_after, idx, dir="left"):
    chi0 = 1 if idx == 0 else dims[idx - 1]
    chi1 = dims[idx]
    chi1p = dims_after[idx]
    chi2 = 1 if idx == len(dims) - 1 else dims[idx + 1]
    if dir == "right":
        chi0, chi2 = chi2, chi0
    return 8*chi0*chi1*chi1p + (80/3) * chi1 * chi2 * min(chi1, 4*chi2)

def compute_flops_cost(braid, printd=False, chi=float('inf')):
    total = 0
    dirs = [None] * braid['strands']
    for (idx, _), dims, dims_after in zip(braid['crossing_list'][1:], braid['mpo-0']['bond_sizes'], braid['mpo-0']['bond_sizes'][1:]):
        dims = [min(d, chi) for d in dims.copy()]
        dims_after = [min(d, chi) for d in dims_after.copy()]

        if printd:
            print("A:", '*'+'*'.join('>' if d == "right" else '<' if d == 'left' else '-' for d in dirs)+'*')

        # Canonicalize around idx
        for j in range(idx):
            if dirs[j] != "right":
                total += canonicalize_cost(dims, idx)
                dirs[j] = "right"
        for j in range(len(dirs) - 1, idx + 1, -1):
            if dirs[j] != "left":
                total += canonicalize_cost(dims, idx)
                dirs[j] = "left"

        if printd:
            print("B:", '*'+'*'.join('>' if d == "right" else '<' if d == 'left' else '-' for d in dirs)+'*')
            
        # Multiply by U
        total += multiply_cost(dims, idx)
        dirs[idx] = None
        dirs[idx + 1] = None
        dims[idx] *= 2
        dims[idx + 1] *= 2

        if printd:
            print("C:", '*'+'*'.join('>' if d == "right" else '<' if d == 'left' else '-' for d in dirs)+'*')

        # Move center to idx
        total += canonicalize_cost(dims, idx + 1)
        dirs[idx + 1] = "left"
        total += canonicalize_cost(dims, idx)
        dirs[idx] = "left"

        if printd:
            print("D:", '*'+'*'.join('>' if d == "right" else '<' if d == 'left' else '-' for d in dirs)+'*')
            
        # Compress bonds rightwards
        total += compress_cost(dims, dims_after, idx, dir="right")
        total += compress_cost(dims, dims_after, idx + 1, dir="right")
        dirs[idx] = "right"
        dirs[idx + 1] = "right"

        if printd:
            print("E:", '*'+'*'.join('>' if d == "right" else '<' if d == 'left' else '-' for d in dirs)+'*')

    # Single sweep to set up for projectors
    for j in range(len(dirs) - 1, -1, -1):
        if dirs[j] != "left":
            total += canonicalize_cost(dims, idx)
            dirs[j] = "left"

    for idx, dims in enumerate(braid['mpo-0']['bond_sizes'][len(braid['crossing_list']) - 1:-1]):
        dims_after = [min(d, chi) for d in dims_after.copy()]
        dims = [min(d, chi) for d in dims.copy()]
        if printd:
            print("F:", '*'+'*'.join('>' if d == "right" else '<' if d == 'left' else '-' for d in dirs)+'*')
            
        # Multiply by proj:
        total += multiply_cost(dims, idx, k=2)
        dirs[idx] = None
        dims[idx] *= 2

        if printd:
            print("G:", '*'+'*'.join('>' if d == "right" else '<' if d == 'left' else '-' for d in dirs)+'*')
            
        # Compress bond rightwards
        total += compress_cost(dims, dims_after, idx, dir="right")
        dirs[idx] = "right"

        if printd:
            print("H:", '*'+'*'.join('>' if d == "right" else '<' if d == 'left' else '-' for d in dirs)+'*')

    # Trace
    dims = braid['mpo-0']['bond_sizes'][-1]
    total += 4 * (dims[0] + dims[-1])
    total += 4 * sum(a * b for a, b in zip(dims, dims[1:]))

    return total


In [28]:
def flops_for_braid(idx):
    braid = braids[idx]
    mchi = max_chi[idx]
    return [compute_flops_cost(braid, chi=factor*mchi) for factor in np.linspace(0, 1, 20)]

if __name__ == "__main__":
    pool = multiprocess.Pool(64) 
    flops = np.array(list(tqdm.tqdm(pool.imap(flops_for_braid, range(len(braids))), total=len(braids))))

100%|██████████| 10000/10000 [00:16<00:00, 613.60it/s]


In [94]:
normalized_flops = np.concatenate([
    np.stack([max_intermediate_chi[:, i] / max_intermediate, flops[:, i] / flops[:, -1]], axis=1)
    for i in range(20)
], axis=0)

In [100]:
normalized_flops_chi = np.concatenate([
    np.stack([np.repeat(j, len(flops[:, -1])), flops[:, i] / flops[:, -1]], axis=1)
    for i, j in enumerate(np.linspace(0, 1, 20))
], axis=0)

In [102]:
np.savez("processed/mpo_props.npz", 
    qubits=qubits, crossings=crossings, depths=depths, abs_errs=abs_errs, rel_errs=rel_errs,
    max_chi=max_chi, max_intermediate=max_intermediate, flops=flops, max_intermediate_chi=max_intermediate_chi, max_intermediate_chi2=max_intermediate_chi2
)

np.savez("processed/mpo_aggregate.npz",
    normalized_flops_im=normalized_flops, normalized_flops_chi=normalized_flops_chi,
    normalized_rel_errs_chi=normalized_rel_errs_capped, normalized_rel_errs_im=normalized_rel_errsa_capped,
)

## Noisy Simulation Data

In [116]:
data_5 = np.load("raw_data/results_small_cross.npz", allow_pickle=True)
data_1 = np.load("raw_data/results_small_1e-4_cross.npz", allow_pickle=True)

In [148]:
crossings = np.array([len(c) for c in data_5['crossings']])
qubits = data_5['qubits']

abs_errs = np.abs(data_5['acs'] - data_5['ms'])
abs_errsn = np.abs(data_5['acs'] - data_5['msn'])
rel_errs = np.abs(data_5['acs'] - data_5['ms']) / np.abs(data_5['acs'])
rel_errsn = np.abs(data_5['acs'] - data_5['msn']) / np.abs(data_5['acs'])

dr = data_5['dr']
gates = data_5['gates']

crossings_data = data_5['crossings']
depths = np.array([calculate_depth(qubits[idx] - 1, [(abs(c) - 1, c/abs(c)) for c in crossings_data[idx]]) for idx in tqdm.trange(len(qubits))])

100%|██████████| 23819/23819 [00:04<00:00, 4855.40it/s]


In [149]:
np.savez("processed/sims_5_props.npz",
    crossings=crossings, qubits=qubits, depths=depths,
    abs_errs=abs_errs, abs_errsn=abs_errsn, rel_errs=rel_errs, rel_errsn=rel_errsn,
    gates=gates, dr=dr
)

In [150]:
crossings = np.array([len(c) for c in data_1['crossings']])
qubits = data_1['qubits']

abs_errs = np.abs(data_1['acs'] - data_1['ms'])
abs_errsn = np.abs(data_1['acs'] - data_1['msn'])
rel_errs = np.abs(data_1['acs'] - data_1['ms']) / np.abs(data_1['acs'])
rel_errsn = np.abs(data_1['acs'] - data_1['msn']) / np.abs(data_1['acs'])

dr = data_1['dr']
gates = data_1['gates']

crossings_data = data_1['crossings']
depths = np.array([calculate_depth(qubits[idx] - 1, [(abs(c) - 1, c/abs(c)) for c in crossings_data[idx]]) for idx in tqdm.trange(len(qubits))])

100%|██████████| 23819/23819 [00:04<00:00, 4968.79it/s]


In [151]:
np.savez("processed/sims_1_props.npz",
    crossings=crossings, qubits=qubits, depths=depths,
    abs_errs=abs_errs, abs_errsn=abs_errsn, rel_errs=rel_errs, rel_errsn=rel_errsn,
    gates=gates, dr=dr
)

## Emulator Data

In [3]:
shots = []
for f in glob.glob(f'device_run/results/h1-1e/normal/*.json'):
    with open(f, 'r') as file:
        dict_results = json.load(file)
        from pytket.backends.backendresult import BackendResult
        res = BackendResult.from_dict(dict_results)
        shots.append(res.get_shots())
shots = np.concatenate(shots, axis=0)

In [5]:
shots_conj = []
for f in glob.glob(f'device_run/results/h1-1e/conj/*.json'):
    with open(f, 'r') as file:
        dict_results = json.load(file)
        from pytket.backends.backendresult import BackendResult
        res = BackendResult.from_dict(dict_results)
        shots_conj.append(res.get_shots())
shots_conj = np.concatenate(shots_conj, axis=0)

In [7]:
bits, data = shots[:, :15], shots[:, 15+16:] 
bits_conj, data_conj = shots_conj[:, :15], shots_conj[:, 15+16:]

In [8]:
def bootstrap_ci(replicas: np.ndarray, alpha: float):
    n = replicas.shape[0]
    if n == 0:
        return (0.0, 0.0)
    elif n == 1:
        return (replicas[0], replicas[0], 0.0)
    alpha2 = scipy.stats.norm.cdf(np.sqrt(n / (n - 1)) * scipy.stats.t.ppf(alpha / 2, n - 1))
    return tuple(np.quantile(replicas, [alpha2, 1.0 - alpha2])) + (np.std(replicas),)

def do_replica(arg):
    i, bits_orig, data_orig, braid, mitigation = arg
    if i == 0:
        data, bits = data_orig, bits_orig
    else:
        indices = np.random.randint(bits_orig.shape[0], size=bits_orig.shape[0])
        data, bits = data_orig[indices, :], bits_orig[indices, :]
        
    mask = (np.all(data[:, 2:] == 0, axis=1) & (data[:, 0] == 0)).astype(float)
    sign = 1 - 2 * data[:, 1].astype(float)
    value = mask * sign

    if mitigation:
        xbits = data[:, 2:] ^ bits[:, 1:]
        reject = (data[:, 0] != 0) | np.any((xbits[:, :-1] | xbits[:, 1:]) == 0, axis=1)
        reals = value[(bits[:, 0] == 0) & ~reject]
        imags = value[(bits[:, 0] == 1) & ~reject]
        dr = np.mean(reject.astype(float), axis=0)
    else:
        reals = value[bits[:, 0] == 0]
        imags = value[bits[:, 0] == 1]
        dr = 0.0

    est_raw = np.mean(reals, axis=0) + 1j * np.mean(imags, axis=0)
    est = (-np.exp(-1j*3*np.pi/5))**(3*braid.writhe) * phi ** (data.shape[-1] - 2) * est_raw
    
    return i, est, est_raw, dr

def compute_jones(bits: np.ndarray, data: np.ndarray, braid: Braid, mitigation: bool, bootstrap: bool, resamples: int = 100, alpha: float = 0.05):
    if bootstrap:
        pbar = tqdm.trange(resamples + 1, desc="processing replicas")
    else:
        pbar = range(1)
        resamples = 0

    ests = [None]*(resamples + 1)
    ests_raw = [None]*(resamples + 1)
    drs = [None]*(resamples + 1)
    for i, est, est_raw, dr in map(do_replica, ((i, bits, data, braid, mitigation) for i in pbar)):
        ests[i] = est
        ests_raw[i] = est_raw
        drs[i] = dr
    est = np.array(ests)
    est_raw = np.array(ests_raw)
    dr = np.array(drs)

    return est[0], bootstrap_ci(est.real, alpha), bootstrap_ci(est.imag, alpha), dr[0], bootstrap_ci(dr, alpha), est_raw[0], bootstrap_ci(est_raw.real, alpha), bootstrap_ci(est_raw.imag, alpha), ests[1:], ests_raw[1:], drs[1:]

In [9]:
res = compute_jones(bits, data, Braid.from_word(json.load(open("device_run/b_n15_l15.json"))['original']['word']), True, True, 10000)
resn = compute_jones(bits, data, Braid.from_word(json.load(open("device_run/b_n15_l15.json"))['original']['word']), False, True, 10000)
res_conj = compute_jones(bits_conj, data_conj, Braid.from_word(json.load(open("device_run/b_n15_l15.json"))['original']['word']), True, True, 10000)
resn_conj = compute_jones(bits_conj, data_conj, Braid.from_word(json.load(open("device_run/b_n15_l15.json"))['original']['word']), False, True, 10000)

processing replicas: 100%|██████████| 10001/10001 [00:02<00:00, 4531.19it/s]


In [12]:
bspec = json.load(open("device_run/b_n15_l15.json"))
braid = Braid.from_word(bspec['original']['word'])
actual_raw = (bspec['jones'][0] + 1j*bspec['jones'][1]) / ((-np.exp(-1j*3*np.pi/5))**(3*(braid.writhe)) * phi ** (data.shape[-1] - 2))
actual_raw

(-0.13274241625389036-0.817078298378928j)

In [15]:
def conj_mitigate(z, zconj, ref):
    z1 = abs(z) * np.sqrt(z / zconj)
    r = abs(z + zconj) / 2
    if z1.real < 0.0:
        r *= -1.0
    i = abs(z - zconj) / 2
    if z1.imag < 0.0:
        i *= -1.0
    z2 = r + 1j * i
    if abs(z2 - ref) > abs(-z2 - ref):
        z2 *= -1.0
    return z2

val_conj_mitigated = conj_mitigate(res[5], res_conj[5], res[5]) 
val_conj_mitigated_replicas = np.array([conj_mitigate(z, zconj, val_conj_mitigated) for z, zconj in zip(res[-2], res_conj[-2])])

val_conj_unmitigated = conj_mitigate(resn[5], resn_conj[5], res[5]) 
val_conj_unmitigated_replicas = np.array([conj_mitigate(z, zconj, val_conj_unmitigated) for z, zconj in zip(resn[-2], resn_conj[-2])])

In [17]:
np.savez("processed/emulator.npz",
    z_mit=res[5], z_mit_reps=res[-2],
    z_unmit=resn[5], z_unmit_reps=resn[-2],
    zc_mit=res_conj[5], zc_mit_reps=res_conj[-2],
    zc_unmit=resn_conj[5], zc_unmit_reps=resn_conj[-2],
    conj_mit=val_conj_mitigated, conj_mit_reps=val_conj_mitigated_replicas,
    conj_unmit=val_conj_unmitigated, conj_unmit_reps=val_conj_unmitigated_replicas,
    actual=actual_raw, writhe=braid.writhe, strands=braid.strands
)